# Types and Values

In Excel you typically organize data in tables where each column is of a a specific *type* like number, text string or date.  In R you use vector, matrix, list and array and your values are of *type* Logical, Numeric, Integer or Character.  Similarly, F# has *types*.

F# has a lot of built-in typs such as `string`, `int` and `decimal` and it also has container types like `array`, `list` and `seq` (sequence) that can contain other items.  The .NET framework itself also has a lot of types that you can use, such as the commonly used `DateTime` type.  

## Records
You have already seen some of the built-in F# types and you have also created some types of your own in the first example, where you created the `PersonPolicy` type.

In [1]:
type PersonPolicy = 
    {
        PersonId: string;
        PolicyNumber: string;
        Premium: decimal;
    }

The below script needs to be able to find the current output cell; this is an easy method to get it.

As you can see, `PersonPolicy` is really just a combination of other types: `string` and `decimal`.  That way you can create an endless number of types by combining existing types.  This kind of type is called a *record* type.  It is also called a *product* type because its sample space is `string * string * decimal`.

You create values of a type by using the `let` keyword.

In [8]:
let theAnswer = 42 // An integer
let greeting = "Hello world" // A string
let pi = 3.141 // A double
let pp = 
    {
        PersonId = "123";
        PolicyNumber = "Pol001";
        Premium = 10000m;
    }

PersonId,PolicyNumber,Premium
123,Pol001,10000


Notice how you don't have to specify the type anywhere.  You just create the value and most of the time F# will figure out what type you intended.  F# will keep track of the types behind the scenes which is very useful when defining functions as you will see later.

> Why the `m`? Notice the `m` after the premium amount in line 8 above?  The `m` tells F# that you want a decimal and not an integer.  If you remove the `m`, you will get an error saying that F# cannot convert the integer 10000 to a decimal, since the Premium field is of type decimal.

## Discriminated Unions
Discriminated unions are a way of defining a type with mutually exclusive ways of creating values of that type.  It sounds weird but it is a really nice way to represent data.  Say for example that you have to policy systems in your company.  One is an old legacy system where policy numbers are represented as integers.  For the other newer system policy numbers are strings.  For this setup you might define the PersonPolicy like so.

In [10]:
type PolicyNumber =
    | LegacyPolicyNumber of int
    | NewPolicyNumber of string
    
type PersonPolicy2 =
    {
        PersonId: string;
        PolicyNumber: PolicyNumber;
        Premium: decimal;
    }    

The `PersonPolicy2` type is used like so.

In [19]:
let legacyPolicyPerson = 
    {
        PersonId = "1";
        PolicyNumber = LegacyPolicyNumber(42);
        Premium = 1000m;
    }
    
let newPolicyPerson =
    {
        PersonId = "2";
        PolicyNumber = NewPolicyNumber("Pol01");
        Premium = 1200m;    
    }

display([ legacyPolicyPerson; newPolicyPerson ])

index,PersonId,PolicyNumber,Premium
0,1,{ FSI_0014+PolicyNumber+LegacyPolicyNumber: Item: 42 },1000
1,2,{ FSI_0014+PolicyNumber+NewPolicyNumber: Item: Pol01 },1200


Using discriminated unions like that gives you complete control and type safety when handling data.  You cannot accidentally use a legacy policy number instead of a new policy number.